In [2]:
%pip install -qU pypdf langchain_community

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import PyPDFLoader

#file_path = "../example_data/nke-10k-2023.pdf"
file_path = "C:/Users/aryye/OneDrive/Documents/GitHub/HLC/data/LCOS Manual.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

467


In [3]:
print(docs[0].page_content[0:100])
print(docs[0].metadata)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
  
 
 
 
 
 
 
 
 
 
LCOS USER MANUAL  
 
 
 
 
 
 
 
 
 
 
 
 
 

{'source': 'C:/Users/aryye/OneDrive/Documents/GitHub/HLC/data/LCOS Manual.pdf', 'page': 0}


In [3]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [4]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

········


In [ ]:
#from langchain.llms import OpenAI
#llm = OpenAI(model_name="text-ada-001", openai_api_key=API_KEY)

In [2]:
%pip install langchain_chroma langchain_openai --user

  Obtaining dependency information for langchain_chroma from https://files.pythonhosted.org/packages/10/05/34b30ff33af5ea7e6e5b6d1bf8ea3a0f2c1462c6b7f750f21dd0179fdf1e/langchain_chroma-0.1.2-py3-none-any.whl.metadata
  Using cached langchain_chroma-0.1.2-py3-none-any.whl.metadata (1.3 kB)
  Obtaining dependency information for chromadb<0.6.0,>=0.4.0 from https://files.pythonhosted.org/packages/55/97/f863c09b2d0cb320869f96e936408bfad1a47b349821256e9a8a229aebe6/chromadb-0.5.4-py3-none-any.whl.metadata
  Using cached chromadb-0.5.4-py3-none-any.whl.metadata (6.8 kB)
  Obtaining dependency information for fastapi<1,>=0.95.2 from https://files.pythonhosted.org/packages/a4/d4/eb78f7c2648a3585095623f207d7e4b85a1be30347e01e0fdcd1d7d167a9/fastapi-0.111.1-py3-none-any.whl.metadata
  Using cached fastapi-0.111.1-py3-none-any.whl.metadata (26 kB)
  Obtaining dependency information for build>=1.0.3 from https://files.pythonhosted.org/packages/e2/03/f3c8ba0a6b6e30d7d18c40faab90807c9bb5e9a1e3b2fe2008

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [15]:
%pip install --user chromadb==0.5.3

In [5]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [9]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "What does LS and  L1 mean?"})
#results = rag_chain.invoke({"input": "What is LCOS? What can you use it for?"})
results

{'input': 'What does LS and  L1 mean?',
 'context': [Document(metadata={'page': 222, 'source': 'C:/Users/aryye/OneDrive/Documents/GitHub/HLC/data/LCOS Manual.pdf'}, page_content='LCOS USER MANUAL   Chapter 14 Page 10 \nCopyright © 2021 Huntington Mark,  LLC. \n \n \nFigure 14-9 a \n \nUse the following table of definitions to assist you.  \nSkills Table  \nTerm Definition  \nL1/G1  Capable of teaching 1 -1 Learning Center \nand/or NCLB students.  \nS1 Critical Reading and Writing  Capable of teaching SAT Critical Reading and \nWriting skills.  \nS1 Math  Capable of teaching SAT Mathematics skills.  \nA1 English and Writing  Capable of teaching ACT English and Writing.  \nA1 Math  Capable of teaching ACT Mathematics skills.  \nE1 Capable of teaching Exam Preparation skills.  \nLS/GS Math (PK -6) Capable of teachi ng 4-1 Learning Center \nand/or NCLB student’s PK -6 grade \nmathematics.  \nLS/GS Math (7 -8) Capable of teaching 4 -1 Learning Center \nand/or NCLB student’s 7 -8 grade \nmat

In [7]:
print(results["context"][0].page_content)

LCOS USER MANUAL


In [8]:
print(results["context"][0].metadata)

{'page': 0, 'source': 'C:/Users/aryye/OneDrive/Documents/GitHub/HLC/data/LCOS Manual.pdf'}
